In [3]:
import pandas as pd

# Cargar el archivo CSV
data_user_genre = pd.read_csv('steam_games.csv')

# Filtrar registros completos con prioridad en 'release_date'
data_user_genre_cleaned = data_user_genre.dropna(subset=['genres', 'year', 'id'])

# Seleccionar solo las columnas 'genres', 'year' y 'id'
data_user_genre_cleaned = data_user_genre_cleaned[['genres', 'year', 'id']]

# Mostrar información del DataFrame resultante
data_user_genre_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28547 entries, 0 to 30066
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   genres  28547 non-null  object 
 1   year    28547 non-null  float64
 2   id      28547 non-null  float64
dtypes: float64(2), object(1)
memory usage: 892.1+ KB


In [6]:
# Genero un explode sobre la columna genres para obtener una fila por genero
import ast

data_user_genre_cleaned['genres'] = data_user_genre_cleaned['genres'].apply(ast.literal_eval)
data_user_genre_cleaned = data_user_genre_cleaned.explode('genres')

In [97]:
data_user_genre_cleaned.head()

,genres,year,id
0,Action,2018,761140.0
0,Casual,2018,761140.0
0,Indie,2018,761140.0
0,Simulation,2018,761140.0
0,Strategy,2018,761140.0


In [64]:
# Convertir la columna 'id' a tipo numérico
data_user_genre_cleaned['id'] = pd.to_numeric(data_user_genre_cleaned['id'], errors='coerce')


In [66]:
result = data_user_genre_cleaned[data_user_genre_cleaned['id'] == 761140]
print(result)



       genres    year        id
0      Action  2018.0  761140.0
0      Casual  2018.0  761140.0
0       Indie  2018.0  761140.0
0  Simulation  2018.0  761140.0
0    Strategy  2018.0  761140.0


In [9]:
import pandas as pd
from ast import literal_eval

# Cargar el archivo CSV
user_items = pd.read_csv('user_items.csv', converters={'items_cleaned': literal_eval})

In [11]:
user_items

,user_id,items_cleaned
0,76561197970982479,"[{'item_id': '10', 'playtime_forever': 6}, {'i..."
1,js41637,"[{'item_id': '10', 'playtime_forever': 0}, {'i..."
2,evcentric,"[{'item_id': '1200', 'playtime_forever': 923},..."
3,Riot-Punch,"[{'item_id': '10', 'playtime_forever': 0}, {'i..."
4,doctr,"[{'item_id': '300', 'playtime_forever': 1131},..."
...,...,...
88305,76561198323066619,"[{'item_id': '413850', 'playtime_forever': 0},..."
88306,76561198326700687,"[{'item_id': '11020', 'playtime_forever': 0}, ..."
88307,XxLaughingJackClown77xX,[]
88308,76561198329548331,"[{'item_id': '304930', 'playtime_forever': 677..."


In [14]:
# Crear una lista para almacenar los DataFrames
dfs = []

# Iterar sobre el DataFrame 'user_items'
for index, row in user_items.iterrows():
    user_id = row['user_id']
    items_cleaned = row['items_cleaned']
    
    # Crear una lista para almacenar las filas de cada usuario
    rows = []

    # Iterar sobre los items de cada usuario
    for item in items_cleaned:
        item_id = item['item_id']
        playtime_forever = item['playtime_forever']  # Nueva línea para obtener 'playtime_forever'
        rows.append({'user_id': user_id, 'item_id': item_id, 'playtime_forever': playtime_forever})  # Incluir 'playtime_forever'

    # Agregar la lista de filas al DataFrame
    temp_df = pd.DataFrame(rows)

    # Agregar el DataFrame temporal a la lista
    dfs.append(temp_df)

# Concatenar todos los DataFrames en uno solo
user_item_genre = pd.concat(dfs, ignore_index=True)

# Guardar el resultado en un archivo CSV
user_item_genre.to_csv('user_item_genre.csv', index=False)


In [108]:
user_item_genre.head()

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0


In [38]:
# Obtener géneros únicos del DataFrame 'data_user_genre_cleaned'
unique_genres = data_user_genre_cleaned['genres'].unique()

# Crear un DataFrame con la columna 'Genre'
unique_genres = pd.DataFrame({'Genre': unique_genres})


In [39]:
unique_genres

,Genre
0,Action
1,Casual
2,Indie
3,Simulation
4,Strategy
5,Free to Play
6,RPG
7,Sports
8,Adventure
9,Racing


In [36]:
import pandas as pd

# Encontrar el user_id con más playtime_forever por cada item_id
max_playtime_users = (
    user_item_genre.groupby(['item_id', 'user_id'])['playtime_forever']
    .sum()
    .reset_index()
)

# Encontrar el índice del máximo valor de playtime_forever para cada item_id
idx = max_playtime_users.groupby('item_id')['playtime_forever'].idxmax()

# Filtrar el DataFrame original usando estos índices
max_playtime_users = max_playtime_users.loc[idx]



In [37]:
max_playtime_users

,item_id,user_id,playtime_forever
7864,10,jiumohoo,356217
14502,100,JohnnyMcShitty,7600
18355,10000,76561197993061389,25524
18430,1002,Baldman,156
18470,10040,76561197970030775,0
...,...,...,...
5083851,99890,76561198056884067,21353
5083972,9990,MortywaN,506
5093165,99900,thiefofrosesinlalaland,369597
5093888,99910,itsvent,91976


In [69]:
# Convertir la columna 'item_id' a tipo numérico
max_playtime_users['item_id'] = pd.to_numeric(max_playtime_users['item_id'], errors='coerce')

In [70]:
result = max_playtime_users[max_playtime_users['item_id'] == 10]
print(result)

      item_id   user_id  playtime_forever
7864       10  jiumohoo            356217


In [71]:
# Obtener una Serie booleana que indica si cada elemento de 'max_playtime_users['item_id']' está en 'data_user_genre_cleaned['id']'
matching_items = max_playtime_users['item_id'].isin(data_user_genre_cleaned['id'])

# Filtrar 'max_playtime_users' para obtener solo las filas donde hay coincidencias
matching_rows = max_playtime_users[matching_items]

# Mostrar las filas coincidentes
print(matching_rows)


         item_id                 user_id  playtime_forever
7864          10                jiumohoo            356217
18430       1002                 Baldman               156
18472     100400       76561198004448633              3620
18670      10090       76561197976652491            203960
22521     100980               jarvgrimr              3612
...          ...                     ...               ...
5081013     9970                -Azsael-               427
5081076    99700       76561197981278253              1468
5082395     9980                mrpfresh              1421
5093165    99900  thiefofrosesinlalaland            369597
5093888    99910                 itsvent             91976

[8627 rows x 3 columns]


In [74]:
# Crear una lista para almacenar los resultados
result_data = []

# Iterar sobre los géneros únicos
for index, genre_row in unique_genres.iterrows():
    genre = genre_row['Genre']
    
    # Filtrar 'data_user_genre_cleaned' para obtener los 'id' para cada género
    genre_ids = data_user_genre_cleaned[data_user_genre_cleaned['genres'] == genre]['id']
    
    # Filtrar 'max_playtime_users' para obtener el 'user_id' que tiene la mayor sumatoria de playtime_forever
    genre_data = max_playtime_users[max_playtime_users['item_id'].isin(genre_ids)]
    
    if not genre_data.empty:
        max_playtime_user = genre_data.groupby('user_id')['playtime_forever'].sum().idxmax()
        
        # Agregar el resultado a la lista
        result_data.append({'Genre': genre, 'User_id': max_playtime_user})
    else:
        # Manejar el caso donde no hay datos para el género actual
        result_data.append({'Genre': genre, 'User_id': None})

# Convertir la lista en un DataFrame
result_df = pd.DataFrame(result_data)




In [99]:
result_df.head()

,Genre,User_id
0,Action,shinomegami
1,Casual,REBAS_AS_F-T
2,Indie,REBAS_AS_F-T
3,Simulation,clawbot44
4,Strategy,shinomegami


In [85]:
data_user_genre_cleaned['year'] = data_user_genre_cleaned['year'].astype(int)
data_user_genre_cleaned.head()


,genres,year,id
0,Action,2018,761140.0
0,Casual,2018,761140.0
0,Indie,2018,761140.0
0,Simulation,2018,761140.0
0,Strategy,2018,761140.0


In [123]:
result_df

,Genre,User_id
0,action,shinomegami
1,casual,rebas_as_f-t
2,indie,rebas_as_f-t
3,simulation,clawbot44
4,strategy,shinomegami
5,free to play,shinomegami
6,rpg,shinomegami
7,sports,76561197995725436
8,adventure,rebas_as_f-t
9,racing,downsyndromekid


In [124]:
data_user_genre_cleaned

,genres,year,id
0,Action,2018,761140
0,Casual,2018,761140
0,Indie,2018,761140
0,Simulation,2018,761140
0,Strategy,2018,761140
...,...,...,...
30065,Indie,2018,610660
30065,Racing,2018,610660
30065,Simulation,2018,610660
30066,Casual,2017,658870


In [125]:
user_item_genre

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0
...,...,...,...
5153204,76561198329548331,346330,0
5153205,76561198329548331,373330,0
5153206,76561198329548331,388490,3
5153207,76561198329548331,521570,4


In [132]:
result = data_user_genre_cleaned[data_user_genre_cleaned['genres'] == 'Action']
print(result)

Empty DataFrame
Columns: [Genre, User_id, horas_year]
Index: []


In [133]:
import pandas as pd

# Convertir 'User_id' en result_df a minúsculas
result_df['User_id'] = result_df['User_id'].str.lower()

# Convertir 'genres' en data_user_genre_cleaned a minúsculas
data_user_genre_cleaned['genres'] = data_user_genre_cleaned['genres'].str.lower()

# Asegurar que 'id' en data_user_genre_cleaned sea del mismo tipo que 'item_id' en user_item_genre
data_user_genre_cleaned['id'] = pd.to_numeric(data_user_genre_cleaned['id'], errors='coerce')

# Asegurar que 'item_id' en user_item_genre sea del mismo tipo que 'id' en data_user_genre_cleaned
user_item_genre['item_id'] = pd.to_numeric(user_item_genre['item_id'], errors='coerce')

# Convertir 'user_id' en user_item_genre a minúsculas
user_item_genre['user_id'] = user_item_genre['user_id'].str.lower()

# Inicializar una lista para almacenar los diccionarios
horas_year_list = []

# Iterar sobre las filas de result_df
for index, row in result_df.iterrows():
    genre = row['Genre'].lower()  # Convertir a minúsculas
    user_id = row['User_id']
    
    # Filtrar data_user_genre_cleaned por Genre (insensible a mayúsculas y minúsculas)
    genre_data = data_user_genre_cleaned[data_user_genre_cleaned['genres'] == genre].copy()
    
    # Filtrar user_item_genre por user_id (insensible a mayúsculas y minúsculas)
    user_data = user_item_genre[user_item_genre['user_id'] == user_id].copy()
    
    # Convertir 'id' y 'item_id' a formato numérico entero
    genre_data['id'] = pd.to_numeric(genre_data['id'], errors='coerce')
    user_data['item_id'] = pd.to_numeric(user_data['item_id'], errors='coerce')
    
    # Fusionar los DataFrames filtrados en base a 'id' e 'item_id'
    merged_data = pd.merge(genre_data, user_data, left_on='id', right_on='item_id', how='inner')
    
    # Verificar si hay resultados después de la fusión
    if not merged_data.empty:
        # Agrupar y sumar las horas por 'year'
        horas_por_año = merged_data.groupby('year')['playtime_forever'].sum().to_dict()
    else:
        # Si no hay coincidencias, establecer un diccionario vacío
        horas_por_año = {}

    # Agregar el diccionario a la lista
    horas_year_list.append(horas_por_año)

# Agregar la lista como una nueva columna en result_df
result_df['horas_year'] = horas_year_list

# Mostrar el resultado
print(result_df)


                        Genre            User_id  \
0                      action        shinomegami   
1                      casual       rebas_as_f-t   
2                       indie       rebas_as_f-t   
3                  simulation          clawbot44   
4                    strategy        shinomegami   
5                free to play        shinomegami   
6                         rpg        shinomegami   
7                      sports  76561197995725436   
8                   adventure       rebas_as_f-t   
9                      racing    downsyndromekid   
10      massively multiplayer        evilutional   
11               early access  76561197978756659   
12   animation &amp; modeling         scottyg555   
13           video production         scottyg555   
14             web publishing  76561198035718256   
15                  education  76561198059330972   
16          software training        lickidactyl   
17                  utilities  76561198073642113   
18  design &

In [138]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Genre       22 non-null     object
 1   User_id     21 non-null     object
 2   horas_year  22 non-null     object
dtypes: object(3)
memory usage: 660.0+ bytes


In [143]:
# Guardar result_df en un archivo CSV llamado 'def_userforgenre.csv'
result_df.to_csv('def_userforgenre.csv', index=False)

In [1]:
import pandas as pd

# Cargar el archivo def_userforgenre.csv
def_userforgenre = pd.read_csv('def_userforgenre.csv')

def UserForGenre(genero: str):
    # Convertir el género a minúsculas
    genero = genero.lower()

    # Filtrar el DataFrame por el género dado
    genre_data = def_userforgenre[def_userforgenre['Genre'].str.lower() == genero]

    if not genre_data.empty:
        # Encontrar el usuario con más horas jugadas
        max_user = genre_data.loc[genre_data['horas_year'].idxmax()]['User_id']

        # Obtener las horas jugadas por año
        horas_por_año = eval(genre_data.loc[genre_data['User_id'] == max_user, 'horas_year'].iloc[0])

        # Construir el resultado final
        resultado = {
            "Usuario con más horas jugadas para Género " + genero.capitalize(): max_user,
            "Horas jugadas": horas_por_año
        }
        return resultado
    else:
        return {"Mensaje": "No se encontraron datos para el género proporcionado."}

# Ejemplo de uso
genero = "action"
resultado = UserForGenre(genero)
print(resultado)


{'Usuario con más horas jugadas para Género Action': 'shinomegami', 'Horas jugadas': {1988: 136, 1990: 0, 1991: 0, 1992: 0, 1993: 0, 1994: 0, 1995: 2, 1996: 1161, 1997: 99, 1998: 3716, 1999: 303, 2000: 10775, 2001: 2429, 2002: 0, 2003: 532756, 2004: 1225, 2005: 343, 2006: 2180, 2007: 68425, 2008: 180, 2009: 213225, 2010: 9588, 2011: 156697, 2012: 20782, 2013: 166380, 2014: 180793, 2015: 166348, 2016: 42750, 2017: 135}}
